# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator import GraphRepresentationGenerator
from dataset_manager import (
    MovieLensManager,
    INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    ROOT,
)
from llm_manager import GraphPrompterHFClassifier

In [2]:
EPOCHS = 1
BATCH_SIZE_KGE = 128000
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[200948] },
  target={
    node_id=[87585],
    x=[87585, 20],
  },
  (source, edge, target)={ edge_index=[2, 32000204] },
  (target, rev_edge, source)={ edge_index=[2, 32000204] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data,
    kg_manager.gnn_train_data,
    kg_manager.gnn_val_data,
    kg_manager.gnn_test_data,
    hidden_channels=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    kge_dimension=INPUT_EMBEDS_REPLACE_KGE_DIMENSION,
    force_recompute=False,
)

loading pretrained model
Device: 'cpu'


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
INPUT_EMBEDS_REPLACE_ROOT = f"{ROOT}/llm/graph_prompter_hf"

In [7]:
graph_prompter_hf_bert_classifier = GraphPrompterHFClassifier(
    kg_manager,
    graph_representation_generator_graph_prompter_hf.get_embeddings,
    root_path=INPUT_EMBEDS_REPLACE_ROOT,
)

Some weights of GraphPrompterHFBertForSequenceClassification were not initialized from the model checkpoint at ./data/llm/graph_prompter_hf/training/best and are newly initialized because the shapes did not match:
- bert.embeddings.token_type_embeddings.weight: found shape torch.Size([6, 128]) in the checkpoint and torch.Size([5, 128]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


5
5
5
device cuda


In [8]:
dataset_embedding = kg_manager.generate_graph_prompter_hf_embedding_dataset(
    graph_prompter_hf_bert_classifier.tokenizer.sep_token,
    graph_prompter_hf_bert_classifier.tokenizer.pad_token,
    graph_prompter_hf_bert_classifier.tokenize_function,
)

Loading dataset from disk:   0%|          | 0/103 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/26 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/26 [00:00<?, ?it/s]

In [9]:
graph_prompter_hf_bert_classifier.train_model_on_data(
    dataset_embedding, epochs=EPOCHS, batch_size=BATCH_SIZE_LLM
)

  0%|          | 0/100001 [00:00<?, ?it/s]

torch.Size([256, 256, 128]) torch.Size([256, 2]) torch.Size([256, 2, 128])


RuntimeError: Index tensor must have the same number of dimensions as self tensor

In [ ]:
graph_representation_generator_graph_prompter_hf.save_model()

In [12]:
import torch

tens_1 = torch.load("./data/gnn/model_128.pth")
tens_2 = torch.load("./data/gnn/model_128 copy.pth")

In [26]:
(
    tens_2["gnn.conv2.source__edge__target.lin_l.weight"]
    == tens_1["conv2.source__edge__target.lin_l.weight"]
).all()

tensor(True)

In [23]:
tens_1.keys()

odict_keys(['conv1.source__edge__target.lin_l.weight', 'conv1.source__edge__target.lin_l.bias', 'conv1.source__edge__target.lin_r.weight', 'conv1.target__rev_edge__source.lin_l.weight', 'conv1.target__rev_edge__source.lin_l.bias', 'conv1.target__rev_edge__source.lin_r.weight', 'conv2.source__edge__target.lin_l.weight', 'conv2.source__edge__target.lin_l.bias', 'conv2.source__edge__target.lin_r.weight', 'conv2.target__rev_edge__source.lin_l.weight', 'conv2.target__rev_edge__source.lin_l.bias', 'conv2.target__rev_edge__source.lin_r.weight'])